In [21]:
import cv2
from wrapper_vpgnet import WrapperVPGNet
import utils
import numpy as np

#model_path = '/home/luben/csie5452-lane-detection/vpgnet_weights_136_8_10_50/vpgnet_epoch_20.pth'
model_path = '/home/luben/csie5452-lane-detection/vpgnet_weights_1346_8_10_100_lr_1e-5/vpgnet_epoch_55.pth'
vpgnet = WrapperVPGNet(model_path)

video_path = '/home/luben/car-data/demo4.mp4'
cap = cv2.VideoCapture(video_path)

color_map_mat = np.zeros((19, 3), dtype=np.uint8)
for i in range(0, 19):
    if i == 1:
        # lane_solid_white
        color_map_mat[i] = (255, 255, 255)
    elif i == 3:
        # lane_double_white
        color_map_mat[i] = (255, 255, 255)
    elif i == 4:
        # lane_solid_yellow
        color_map_mat[i] = (0, 255, 255)
    elif i == 6:
        #lane_double_yellow
        color_map_mat[i] = (0, 255, 255)
    elif i == 18:
        # erfnet_mask
        color_map_mat[i] = (0, 0, 255)
            
def vpgnet_inference(image_origin):
    
    obj_mask_pred_160x120_vpgnet, vp_mask = vpgnet.get_lane_line_from_image_640x480(image_origin)
    lane_solid_white = utils.resize_array(obj_mask_pred_160x120_vpgnet[0, :, :], (image_origin.shape[0], image_origin.shape[1]))
    lane_double_white = utils.resize_array(obj_mask_pred_160x120_vpgnet[1, :, :], (image_origin.shape[0], image_origin.shape[1]))
    lane_solid_yellow = utils.resize_array(obj_mask_pred_160x120_vpgnet[2, :, :], (image_origin.shape[0], image_origin.shape[1]))
    lane_double_yellow = utils.resize_array(obj_mask_pred_160x120_vpgnet[3, :, :], (image_origin.shape[0], image_origin.shape[1]))

    threshold = 0.95
    lane_solid_white = np.where(lane_solid_white > threshold, 1, 0)
    lane_double_white = np.where(lane_double_white > threshold, 1, 0)
    lane_solid_yellow = np.where(lane_solid_yellow > threshold, 1, 0)
    lane_double_yellow = np.where(lane_double_yellow > threshold, 1, 0)

    
    prob_map_lane_solid_white = lane_solid_white * 1
    prob_map_lane_double_white = lane_double_white * 3
    prob_map_lane_solid_yellow = lane_solid_yellow * 4
    prob_map_lane_double_yellow = lane_double_yellow * 6

    seg_image_lane_solid_white = color_map_mat[prob_map_lane_solid_white]
    seg_image_lane_double_white = color_map_mat[prob_map_lane_double_white]
    seg_image_lane_solid_yellow = color_map_mat[prob_map_lane_solid_yellow]
    seg_image_lane_double_yellow = color_map_mat[prob_map_lane_double_yellow]

    gamma = 0
    img_lane_all = cv2.addWeighted(seg_image_lane_double_yellow, 1, seg_image_lane_solid_white, 1, gamma)
    img_lane_all = cv2.addWeighted(seg_image_lane_solid_yellow, 1, img_lane_all, 1, gamma)
    img_lane_all = cv2.addWeighted(seg_image_lane_double_white, 1, img_lane_all, 1, gamma)
    img_lane_all = cv2.blur(img_lane_all, (9, 9))
    img_lane_all = cv2.addWeighted(image_origin, 0.8, img_lane_all, 0.7, gamma)
    
    return img_lane_all

while cap.isOpened():

    ret, frame = cap.read()
    if ret:
        frame = cv2.resize(frame, (640, 480))
        result_frame = vpgnet_inference(frame)
        cv2.imshow('frame',result_frame)
    if not ret:
        print('can\'t receive frame')
        break
    
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

can't receive frame


In [22]:
import cv2
import numpy as np
from wrapper_vpgnet import WrapperVPGNet
import utils
from matplotlib import pyplot as plt

#model_path = '/home/luben/csie5452-lane-detection/vpgnet_weights/vpgnet_146_455_01.pth'
model_path = '/home/luben/csie5452-lane-detection/vpgnet_weights_136_8_10_50/vpgnet_epoch_20.pth'

vpgnet = WrapperVPGNet(model_path)
#image_origin = cv2.imread('demo/scene005.jpg', cv2.IMREAD_COLOR)
image_origin = cv2.imread('/home/luben/car-data/car1.jpg', cv2.IMREAD_COLOR)

copy_image_origin_vpgnet = cv2.resize(image_origin.copy(), (640, 480))
obj_mask_pred_160x120_vpgnet, vp_mask = vpgnet.get_lane_line_from_image_640x480(copy_image_origin_vpgnet)



# obj_mask_pred_1920x1080_vpgnet = np.resize(obj_mask_pred_160x120_vpgnet, (4, 1080, 1920))

#lane_pred = utils.resize_array(obj_mask_pred_160x120_vpgnet[0, :, :], (939, 1921))
lane_solid_white = utils.resize_array(obj_mask_pred_160x120_vpgnet[0, :, :], (image_origin.shape[0], image_origin.shape[1]))
lane_double_white = utils.resize_array(obj_mask_pred_160x120_vpgnet[1, :, :], (image_origin.shape[0], image_origin.shape[1]))
lane_double_yellow = utils.resize_array(obj_mask_pred_160x120_vpgnet[2, :, :], (image_origin.shape[0], image_origin.shape[1]))

threshold = 0.95

lane_solid_white = np.where(lane_solid_white > threshold, 1, 0)
lane_double_white = np.where(lane_double_white > threshold, 1, 0)
lane_double_yellow = np.where(lane_double_yellow > threshold, 1, 0)

'''
mask = cv2.imread('mask.jpg')
#print(mask.shape)
#print(mask.nonzero())
plt.figure(figsize=(10,5))
plt.axis('off')
plt.imshow(mask)
plt.show()
'''

'''
plt.figure(figsize=(10,5))
plt.axis('off')
plt.imshow(image_origin[:,:,::-1])
plt.show()
'''


color_map_mat = np.zeros((19, 3), dtype=np.uint8)
for i in range(0, 19):
    if i == 1:
        # lane_solid_white
        color_map_mat[i] = (255, 255, 255)
    elif i == 3:
        # lane_double_white
        color_map_mat[i] = (255, 255, 255)
    elif i == 6:
        #lane_double_yellow
        color_map_mat[i] = (255, 255, 0)
    elif i == 18:
        # erfnet_mask
        color_map_mat[i] = (0, 0, 255)

prob_map_lane_solid_white = lane_solid_white * 1
prob_map_lane_double_white = lane_double_white * 3
prob_map_lane_double_yellow = lane_double_yellow * 6

seg_image_lane_solid_white = color_map_mat[prob_map_lane_solid_white]
seg_image_lane_double_white = color_map_mat[prob_map_lane_double_white]
seg_image_lane_double_yellow = color_map_mat[prob_map_lane_double_yellow]

gamma = 0
img_lane_all = cv2.addWeighted(seg_image_lane_double_white, 1, seg_image_lane_solid_white, 1, gamma)
img_lane_all = cv2.addWeighted(seg_image_lane_double_yellow, 1, img_lane_all, 1, gamma)
img_lane_all = cv2.blur(img_lane_all, (9, 9))
img_lane_all = cv2.addWeighted(image_origin[:,:,::-1], 0.8, img_lane_all, 0.7, gamma)

plt.figure(figsize=(10,5))
plt.axis('off')
plt.imshow(img_lane_all)
plt.show()


RuntimeError: Error(s) in loading state_dict for VPGNet:
	size mismatch for obj_mask.4.weight: copying a param with shape torch.Size([256, 3, 2, 2]) from checkpoint, the shape in current model is torch.Size([256, 4, 2, 2]).
	size mismatch for obj_mask.4.bias: copying a param with shape torch.Size([3]) from checkpoint, the shape in current model is torch.Size([4]).

In [ ]:
print('lane_solid_white')
plt.figure(figsize=(10,5))
plt.axis('off')
plt.imshow(lane_solid_white, cmap='gray')
plt.show()
print(lane_solid_white)
print('lane_double_white')
plt.figure(figsize=(10,5))
plt.axis('off')
plt.imshow(lane_double_white, cmap='gray')
plt.show()

print('lane_double_yellow')
plt.figure(figsize=(10,5))
plt.axis('off')
plt.imshow(lane_double_yellow, cmap='gray')
plt.show()

## vanish point  (not working)

In [ ]:
threshold_vp = 0.9999
vp_mask_filter = np.where(vp_mask > threshold_vp, 1, 0)

plt.figure(figsize=(10,5))
plt.axis('off')
plt.imshow(vp_mask_filter, cmap='gray')
plt.show()

## find lane line

In [ ]:
minLineLength = 100
maxLineGap = 10

solid_white = lane_solid_white.copy().astype(np.uint8)
double_white = lane_double_white.copy().astype(np.uint8)
double_yellow = lane_double_yellow.copy().astype(np.uint8)

lines_solid_white = cv2.HoughLinesP(solid_white, 1, np.pi / 180, 100, minLineLength, maxLineGap)
lines_double_white = cv2.HoughLinesP(double_white, 1, np.pi / 180, 100, minLineLength, maxLineGap)
lines_double_yellow = cv2.HoughLinesP(double_yellow, 1, np.pi / 180, 100, minLineLength, maxLineGap)

image = img_lane_all.copy()

if isinstance(lines_solid_white,np.ndarray):    
    for x1, y1, x2, y2 in lines_solid_white[0]:
        cv2.line(image, (x1, y1), (x2, y2), (0, 0, 0), 12)
        cv2.line(image, (x1, y1), (x2, y2), (255, 255, 255), 8)
    '''
    for x1, y1, x2, y2 in lines_solid_white[6]:
        cv2.line(image, (x1, y1), (x2, y2), (0, 0, 0), 5)
        cv2.line(image, (x1, y1), (x2, y2), (255, 255, 255), 4)
    '''
'''
if isinstance(lines_double_white,np.ndarray):
    for x1, y1, x2, y2 in lines_double_white[0]:
        cv2.line(image, (x1, y1), (x2, y2), (0, 0, 0), 5)
        cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 4)
'''
if isinstance(lines_double_yellow,np.ndarray):
    for x1, y1, x2, y2 in lines_double_yellow[0]:
        cv2.line(image, (x1, y1), (x2, y2), (0, 0, 0), 12)
        cv2.line(image, (x1, y1), (x2, y2), (230, 230, 0), 8)



plt.figure(figsize=(10,5))
plt.axis('off')
plt.imshow(image)
plt.show()

In [ ]:
a = np.array([1,2,3])
b = None
for itm in [a,b]:
    c= isinstance(itm,np.ndarray)
    print(c)

In [ ]:
import torch
from torch import nn
m = nn.Sigmoid()
loss = nn.BCELoss()
input = torch.randn(3,3, requires_grad=True)
print(input)
target = torch.empty(3,3).random_(2)
print(target)

a = loss(m(input[0]), target[0])
b = loss(m(input[1]), target[1])
c = loss(m(input[2]), target[2])

print(a,b,c)
print((a+b+c)/3)
output = loss(m(input), target)
print(output)
output.backward()